$\epsilon_{ijk}$


In [1]:
from itertools import cycle
import numpy as np
from sympy.utilities.iterables import multiset_permutations
from sympy.functions.special.tensor_functions import LeviCivita

In [9]:
class EvenPermute:
    
    def __init__(self,array):
        self.a = array
        self.permutes = list()
                
    def __iter__(self):

        for p in multiset_permutations(self.a):
            if (LeviCivita(*p) == 1):
                self.permutes.append(p)
        self.permutes = cycle(self.permutes)
        return self
    def __next__(self):
        return next(self.permutes)
    
class CyclicPermute:
    
    def __init__(self,array,shift=1,it_limit=0):
        self.cnt = len(array)
        if (it_limit and (it_limit < self.cnt)):
            self.cnt = it_limit
        self.permutes = np.array(array)
                
    def __iter__(self):
        return self
    
    def __next__(self):
        a = self.permutes
        if (self.cnt == 0):
            raise StopIteration
        self.cnt -= 1
        
        self.permutes = np.roll(self.permutes,1)
        return a
    
class MiddleCyclicPermute:
    
    def __init__(self,array,shift=1):
        array = np.array(array)
        self.cyc = iter(CyclicPermute(array[1:-1])) #chop off beginning and end
        self.head = array[:1]
        self.tail = array[-1:]
                
    def __iter__(self):
        return self
    
    def __next__(self):
        a = np.concatenate((self.head,next(self.cyc),self.tail))
        return a
        
        
bgp = CyclicPermute([0,1,2,3,6],it_limit=1)
bgp = iter(bgp)

for p in bgp:
     print(p)

[0 1 2 3 6]


In [722]:
np.concatenate?

In [10]:
pal = np.array([[0,1,2,3]])
pal_it = iter(CyclicPermute(pal))

method = lambda x: CyclicPermute(x,it_limit=1)
# method = MiddleCyclicPermute

# for s in multiset_permutations((0,1,2,3,4,5,6)):
#     z_0 = s
z_0  = [0,1,2,3,4,5,6]
z_it = iter(method(z_0))

#choose a starting x,y which map to certain starting palette p_x_n, p_y_n respectively (I guess x,y /are/ the st. palette)
x_n = np.zeros(len(z_0))
y_n = np.zeros(len(z_0))

d = len(z_0)
#choose permutation of x_0, with all possible values for it
x_0 = np.array(np.meshgrid((*[pal for x in np.arange(d)]))).T.reshape(-1,d)
y_0 = np.zeros(x_0.shape) #preallocate
#determine what y_0 goes with that, so that x+y=z 
y_0 = z_0 - x_0

#values of y have to all be between 0,3     
idx = np.all(y_0 >= 0,axis=1) & np.all(y_0 <= 3,axis=1) & np.all(x_0 >= 0,axis=1) & np.all(x_0 <= 3,axis=1) 
y_0 = y_0[np.ix_(idx)]
x_0 = x_0[np.ix_(idx)] 

# for x,y in zip(x_0,y_0):
#     assert(np.all(x+y==z_0))
print('x0 combinations:' ,len(x_0))

#generate a list of possible palette permutations
d=len(pal[0])
gen = np.array(np.meshgrid((*[pal for x in np.arange(d)]))).T.reshape(-1,d)

palettes = np.zeros((len(gen)*len(gen),2,len(pal[0]) ),dtype='int')
cnt=0
for i,p1 in enumerate(gen):
    for j, p2 in enumerate(gen):
        #p1
        palettes[cnt][0][:] = p1
        #p2
        palettes[cnt][1][:] = p2
        cnt+=1

#### Method 2

i = np.arange(palettes.shape[0])[:,None]
j = np.arange(palettes.shape[1])[:,None,None]
k = x_0[:,None]
px = palettes[:,0,x_0]
py = palettes[:,1,y_0]

z_it = iter(method(z_0)) #back to start of cycle
matches=[]
for z_n in z_it:
    idx_xn = np.all(px + py == z_n,axis=2)
    matches.append(idx_xn)
    #idx_palettes = np.any(idx_xn,axis=1)

arr = np.array((matches))
arr.shape

#determine overlap between iterations, i.e. which indices of sequence of numbers are present each time
from functools import reduce
wheres = [np.where(a)[1] for a in matches]
reduced = reduce(np.intersect1d, wheres)
print('sequence', z_0)
print('solutions found:', len(reduced))

for r in reduced:
    x,y = x_0[r],y_0[r]

    px = palettes[:,0,[x]]
    py = palettes[:,1,[y]]
    ps = []

    z_it = iter(method(z_0)) #back to start of cycle
    for z_n in z_it:
        idx_xn = np.all(px+py==z_n,axis=2)
        ps.append(np.where(np.any(idx_xn,axis=1))[0])

    print('x:', x)
    print('y:', y)
    print('pal:')
    for p in ps:
        pal = palettes[p[0]]#np.ix_(p)]
        print(pal)
        print(pal[0][x] + pal[1][y])
#         print('x...')

x0 combinations: 144
sequence [0, 1, 2, 3, 4, 5, 6]
solutions found: 528
x: [0 0 0 0 3 3 3]
y: [0 1 2 3 1 2 3]
pal:
[[0 0 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 3 3 3 3]
y: [0 1 2 0 1 2 3]
pal:
[[0 0 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 0 0 1 3 3]
y: [0 0 2 3 3 2 3]
pal:
[[0 1 0 3]
 [0 0 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 0 1 1 3 3]
y: [0 0 2 2 3 2 3]
pal:
[[0 1 0 3]
 [0 0 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 0 3 1 3 3]
y: [0 0 2 0 3 2 3]
pal:
[[0 1 0 3]
 [0 0 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 0 1 3 3]
y: [0 1 2 3 3 2 3]
pal:
[[0 1 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 0 0 1 3 3]
y: [0 0 2 3 3 2 3]
pal:
[[0 1 0 3]
 [0 0 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 1 0 1 3 3]
y: [0 1 1 3 3 2 3]
pal:
[[0 1 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 1 0 1 3 3]
y: [0 0 1 3 3 2 3]
pal:
[[0 1 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 1 1 3 3]
y: [0 1 2 2 3 2 3]
pal:
[[0 1 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 0 1 1 3 3]
y: [0 0 2 2 3 2 3]
pal:
[[0 1 0 3]
 [0 0 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 1 1 1 3 3]
y: [0 

x: [0 0 2 0 3 2 3]
y: [0 1 0 3 1 3 3]
pal:
[[0 0 2 3]
 [0 1 0 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 1 3 2 3]
y: [0 1 0 2 1 3 3]
pal:
[[0 0 2 3]
 [0 1 3 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 2 3 2 3]
y: [0 1 0 1 1 3 3]
pal:
[[0 0 2 3]
 [0 1 0 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 3 3 2 3]
y: [0 1 0 0 1 3 3]
pal:
[[0 0 2 3]
 [0 1 0 3]]
[0 1 2 3 4 5 6]
x: [0 1 2 0 1 2 3]
y: [0 0 0 3 3 3 3]
pal:
[[0 1 2 3]
 [0 0 0 3]]
[0 1 2 3 4 5 6]
x: [0 1 2 3 1 2 3]
y: [0 0 0 0 3 3 3]
pal:
[[0 1 2 3]
 [0 0 0 3]]
[0 1 2 3 4 5 6]
x: [0 0 1 0 1 2 3]
y: [0 1 1 3 3 3 3]
pal:
[[0 1 2 3]
 [0 1 0 3]]
[0 1 2 3 4 5 6]
x: [0 1 1 0 1 2 3]
y: [0 0 1 3 3 3 3]
pal:
[[0 1 2 3]
 [0 1 0 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 0 1 2 3]
y: [0 1 0 3 3 3 3]
pal:
[[0 1 2 3]
 [0 1 0 3]]
[0 1 2 3 4 5 6]
x: [0 1 2 0 1 2 3]
y: [0 0 0 3 3 3 3]
pal:
[[0 1 2 3]
 [0 0 0 3]]
[0 1 2 3 4 5 6]
x: [0 0 1 2 1 2 3]
y: [0 1 1 1 3 3 3]
pal:
[[0 1 2 3]
 [0 1 0 3]]
[0 1 2 3 4 5 6]
x: [0 1 1 2 1 2 3]
y: [0 0 1 1 3 3 3]
pal:
[[0 1 2 3]
 [0 1 0 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 2 1 2 

x: [0 0 0 1 2 2 3]
y: [0 1 2 2 2 3 3]
pal:
[[0 1 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 0 1 2 2 3]
y: [0 0 2 2 2 3 3]
pal:
[[0 1 2 3]
 [0 0 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 1 1 2 2 3]
y: [0 1 1 2 2 3 3]
pal:
[[0 1 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 1 1 2 2 3]
y: [0 0 1 2 2 3 3]
pal:
[[0 1 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 1 2 2 3]
y: [0 1 0 2 2 3 3]
pal:
[[0 1 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 2 1 2 2 3]
y: [0 0 0 2 2 3 3]
pal:
[[0 1 2 3]
 [0 0 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 2 2 2 3]
y: [0 1 2 1 2 3 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 0 2 2 2 3]
y: [0 0 2 1 2 3 3]
pal:
[[0 1 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 1 2 2 2 3]
y: [0 1 1 1 2 3 3]
pal:
[[0 1 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 1 2 2 2 3]
y: [0 0 1 1 2 3 3]
pal:
[[0 1 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 2 2 2 3]
y: [0 1 0 1 2 3 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 2 2 2 2 3]
y: [0 0 0 1 2 3 3]
pal:
[[0 1 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 3 2 2 

x: [0 1 1 2 3 3 3]
y: [0 0 1 1 1 2 3]
pal:
[[0 1 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 2 3 3 3]
y: [0 1 0 1 1 2 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 2 2 3 3 3]
y: [0 0 0 1 1 2 3]
pal:
[[0 1 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 3 3 3 3]
y: [0 1 2 0 1 2 3]
pal:
[[0 0 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 0 3 3 3 3]
y: [0 0 2 0 1 2 3]
pal:
[[0 1 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 1 3 3 3 3]
y: [0 1 1 0 1 2 3]
pal:
[[0 1 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 1 3 3 3 3]
y: [0 0 1 0 1 2 3]
pal:
[[0 1 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 3 3 3 3]
y: [0 1 0 0 1 2 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 2 3 3 3 3]
y: [0 0 0 0 1 2 3]
pal:
[[0 1 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 0 0 1 2 3]
y: [0 0 2 3 3 3 3]
pal:
[[0 1 2 3]
 [0 0 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 2 0 1 2 3]
y: [0 0 0 3 3 3 3]
pal:
[[0 1 2 3]
 [0 0 0 3]]
[0 1 2 3 4 5 6]
x: [0 1 0 1 1 2 3]
y: [0 0 2 2 3 3 3]
pal:
[[0 1 2 3]
 [0 0 2 3]]
[0 1 2 3 4 5 6]
x: [0 1 2 1 1 2 

x: [0 0 2 3 3 2 3]
y: [0 1 0 0 1 3 3]
pal:
[[0 0 2 3]
 [0 1 0 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 0 2 3 3]
y: [0 1 2 3 2 2 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 0 2 3 3]
y: [0 1 0 3 2 2 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 2 2 3 3]
y: [0 1 2 1 2 2 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 2 2 3 3]
y: [0 1 0 1 2 2 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 3 2 3 3]
y: [0 1 2 0 2 2 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 3 2 3 3]
y: [0 1 0 0 2 2 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 0 3 3 3]
y: [0 1 2 3 1 2 3]
pal:
[[0 0 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 0 3 3 3]
y: [0 1 0 3 1 2 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 2 3 3 3]
y: [0 1 2 1 1 2 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 2 3 3 3]
y: [0 1 0 1 1 2 3]
pal:
[[0 0 2 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 0 3 3 3 3]
y: [0 1 2 0 1 2 3]
pal:
[[0 0 0 3]
 [0 1 2 3]]
[0 1 2 3 4 5 6]
x: [0 0 2 3 3 3 

In [798]:
for r in reduced[:1]:
    x,y = x_0[r],y_0[r]

    px = palettes[:,0,[x]]
    py = palettes[:,1,[y]]
    ps = []
    
    z_it = iter(method(z_0)) #back to start of cycle
    for z_n in z_it:
        idx_xn = np.all(px+py==z_n,axis=2)
        ps.append(np.where(np.any(idx_xn,axis=1))[0])

    print('x:', x)
    print('y:', y)
    print('pal:')
    print(ps)
    for p in ps:
        
        pal = palettes[p[0]]#np.ix_(p)]
        
        print(pal[0][x] + pal[1][y])

x: [0 2 1 3 2]
y: [1 0 2 1 3]
pal:
[array([57585, 57825, 58065, 58305]), array([ 6371,  6611,  6851, 23778, 24018, 24258]), array([10334, 10574, 14362, 14602, 27741, 27981, 31769, 32009]), array([ 6781,  7021, 10809, 11049, 24188, 24428, 28216, 28456]), array([49534, 49774, 50014, 53562, 53802, 54042])]
[1 2 3 4 5]
[5 1 2 3 4]
[4 5 1 2 3]
[3 4 5 1 2]
[2 3 4 5 1]


In [813]:
for s in multiset_permutations((0,1,2,3,4,5,6)):
    print(s)

[0, 1, 2, 3, 4, 5, 6]
[0, 1, 2, 3, 4, 6, 5]
[0, 1, 2, 3, 5, 4, 6]
[0, 1, 2, 3, 5, 6, 4]
[0, 1, 2, 3, 6, 4, 5]
[0, 1, 2, 3, 6, 5, 4]
[0, 1, 2, 4, 3, 5, 6]
[0, 1, 2, 4, 3, 6, 5]
[0, 1, 2, 4, 5, 3, 6]
[0, 1, 2, 4, 5, 6, 3]
[0, 1, 2, 4, 6, 3, 5]
[0, 1, 2, 4, 6, 5, 3]
[0, 1, 2, 5, 3, 4, 6]
[0, 1, 2, 5, 3, 6, 4]
[0, 1, 2, 5, 4, 3, 6]
[0, 1, 2, 5, 4, 6, 3]
[0, 1, 2, 5, 6, 3, 4]
[0, 1, 2, 5, 6, 4, 3]
[0, 1, 2, 6, 3, 4, 5]
[0, 1, 2, 6, 3, 5, 4]
[0, 1, 2, 6, 4, 3, 5]
[0, 1, 2, 6, 4, 5, 3]
[0, 1, 2, 6, 5, 3, 4]
[0, 1, 2, 6, 5, 4, 3]
[0, 1, 3, 2, 4, 5, 6]
[0, 1, 3, 2, 4, 6, 5]
[0, 1, 3, 2, 5, 4, 6]
[0, 1, 3, 2, 5, 6, 4]
[0, 1, 3, 2, 6, 4, 5]
[0, 1, 3, 2, 6, 5, 4]
[0, 1, 3, 4, 2, 5, 6]
[0, 1, 3, 4, 2, 6, 5]
[0, 1, 3, 4, 5, 2, 6]
[0, 1, 3, 4, 5, 6, 2]
[0, 1, 3, 4, 6, 2, 5]
[0, 1, 3, 4, 6, 5, 2]
[0, 1, 3, 5, 2, 4, 6]
[0, 1, 3, 5, 2, 6, 4]
[0, 1, 3, 5, 4, 2, 6]
[0, 1, 3, 5, 4, 6, 2]
[0, 1, 3, 5, 6, 2, 4]
[0, 1, 3, 5, 6, 4, 2]
[0, 1, 3, 6, 2, 4, 5]
[0, 1, 3, 6, 2, 5, 4]
[0, 1, 3, 6, 4, 2, 5]
[0, 1, 3, 

[1, 5, 2, 3, 0, 6, 4]
[1, 5, 2, 3, 4, 0, 6]
[1, 5, 2, 3, 4, 6, 0]
[1, 5, 2, 3, 6, 0, 4]
[1, 5, 2, 3, 6, 4, 0]
[1, 5, 2, 4, 0, 3, 6]
[1, 5, 2, 4, 0, 6, 3]
[1, 5, 2, 4, 3, 0, 6]
[1, 5, 2, 4, 3, 6, 0]
[1, 5, 2, 4, 6, 0, 3]
[1, 5, 2, 4, 6, 3, 0]
[1, 5, 2, 6, 0, 3, 4]
[1, 5, 2, 6, 0, 4, 3]
[1, 5, 2, 6, 3, 0, 4]
[1, 5, 2, 6, 3, 4, 0]
[1, 5, 2, 6, 4, 0, 3]
[1, 5, 2, 6, 4, 3, 0]
[1, 5, 3, 0, 2, 4, 6]
[1, 5, 3, 0, 2, 6, 4]
[1, 5, 3, 0, 4, 2, 6]
[1, 5, 3, 0, 4, 6, 2]
[1, 5, 3, 0, 6, 2, 4]
[1, 5, 3, 0, 6, 4, 2]
[1, 5, 3, 2, 0, 4, 6]
[1, 5, 3, 2, 0, 6, 4]
[1, 5, 3, 2, 4, 0, 6]
[1, 5, 3, 2, 4, 6, 0]
[1, 5, 3, 2, 6, 0, 4]
[1, 5, 3, 2, 6, 4, 0]
[1, 5, 3, 4, 0, 2, 6]
[1, 5, 3, 4, 0, 6, 2]
[1, 5, 3, 4, 2, 0, 6]
[1, 5, 3, 4, 2, 6, 0]
[1, 5, 3, 4, 6, 0, 2]
[1, 5, 3, 4, 6, 2, 0]
[1, 5, 3, 6, 0, 2, 4]
[1, 5, 3, 6, 0, 4, 2]
[1, 5, 3, 6, 2, 0, 4]
[1, 5, 3, 6, 2, 4, 0]
[1, 5, 3, 6, 4, 0, 2]
[1, 5, 3, 6, 4, 2, 0]
[1, 5, 4, 0, 2, 3, 6]
[1, 5, 4, 0, 2, 6, 3]
[1, 5, 4, 0, 3, 2, 6]
[1, 5, 4, 0, 3, 6, 2]
[1, 5, 4, 

[3, 0, 4, 6, 5, 2, 1]
[3, 0, 5, 1, 2, 4, 6]
[3, 0, 5, 1, 2, 6, 4]
[3, 0, 5, 1, 4, 2, 6]
[3, 0, 5, 1, 4, 6, 2]
[3, 0, 5, 1, 6, 2, 4]
[3, 0, 5, 1, 6, 4, 2]
[3, 0, 5, 2, 1, 4, 6]
[3, 0, 5, 2, 1, 6, 4]
[3, 0, 5, 2, 4, 1, 6]
[3, 0, 5, 2, 4, 6, 1]
[3, 0, 5, 2, 6, 1, 4]
[3, 0, 5, 2, 6, 4, 1]
[3, 0, 5, 4, 1, 2, 6]
[3, 0, 5, 4, 1, 6, 2]
[3, 0, 5, 4, 2, 1, 6]
[3, 0, 5, 4, 2, 6, 1]
[3, 0, 5, 4, 6, 1, 2]
[3, 0, 5, 4, 6, 2, 1]
[3, 0, 5, 6, 1, 2, 4]
[3, 0, 5, 6, 1, 4, 2]
[3, 0, 5, 6, 2, 1, 4]
[3, 0, 5, 6, 2, 4, 1]
[3, 0, 5, 6, 4, 1, 2]
[3, 0, 5, 6, 4, 2, 1]
[3, 0, 6, 1, 2, 4, 5]
[3, 0, 6, 1, 2, 5, 4]
[3, 0, 6, 1, 4, 2, 5]
[3, 0, 6, 1, 4, 5, 2]
[3, 0, 6, 1, 5, 2, 4]
[3, 0, 6, 1, 5, 4, 2]
[3, 0, 6, 2, 1, 4, 5]
[3, 0, 6, 2, 1, 5, 4]
[3, 0, 6, 2, 4, 1, 5]
[3, 0, 6, 2, 4, 5, 1]
[3, 0, 6, 2, 5, 1, 4]
[3, 0, 6, 2, 5, 4, 1]
[3, 0, 6, 4, 1, 2, 5]
[3, 0, 6, 4, 1, 5, 2]
[3, 0, 6, 4, 2, 1, 5]
[3, 0, 6, 4, 2, 5, 1]
[3, 0, 6, 4, 5, 1, 2]
[3, 0, 6, 4, 5, 2, 1]
[3, 0, 6, 5, 1, 2, 4]
[3, 0, 6, 5, 1, 4, 2]
[3, 0, 6, 

[4, 2, 6, 3, 1, 0, 5]
[4, 2, 6, 3, 1, 5, 0]
[4, 2, 6, 3, 5, 0, 1]
[4, 2, 6, 3, 5, 1, 0]
[4, 2, 6, 5, 0, 1, 3]
[4, 2, 6, 5, 0, 3, 1]
[4, 2, 6, 5, 1, 0, 3]
[4, 2, 6, 5, 1, 3, 0]
[4, 2, 6, 5, 3, 0, 1]
[4, 2, 6, 5, 3, 1, 0]
[4, 3, 0, 1, 2, 5, 6]
[4, 3, 0, 1, 2, 6, 5]
[4, 3, 0, 1, 5, 2, 6]
[4, 3, 0, 1, 5, 6, 2]
[4, 3, 0, 1, 6, 2, 5]
[4, 3, 0, 1, 6, 5, 2]
[4, 3, 0, 2, 1, 5, 6]
[4, 3, 0, 2, 1, 6, 5]
[4, 3, 0, 2, 5, 1, 6]
[4, 3, 0, 2, 5, 6, 1]
[4, 3, 0, 2, 6, 1, 5]
[4, 3, 0, 2, 6, 5, 1]
[4, 3, 0, 5, 1, 2, 6]
[4, 3, 0, 5, 1, 6, 2]
[4, 3, 0, 5, 2, 1, 6]
[4, 3, 0, 5, 2, 6, 1]
[4, 3, 0, 5, 6, 1, 2]
[4, 3, 0, 5, 6, 2, 1]
[4, 3, 0, 6, 1, 2, 5]
[4, 3, 0, 6, 1, 5, 2]
[4, 3, 0, 6, 2, 1, 5]
[4, 3, 0, 6, 2, 5, 1]
[4, 3, 0, 6, 5, 1, 2]
[4, 3, 0, 6, 5, 2, 1]
[4, 3, 1, 0, 2, 5, 6]
[4, 3, 1, 0, 2, 6, 5]
[4, 3, 1, 0, 5, 2, 6]
[4, 3, 1, 0, 5, 6, 2]
[4, 3, 1, 0, 6, 2, 5]
[4, 3, 1, 0, 6, 5, 2]
[4, 3, 1, 2, 0, 5, 6]
[4, 3, 1, 2, 0, 6, 5]
[4, 3, 1, 2, 5, 0, 6]
[4, 3, 1, 2, 5, 6, 0]
[4, 3, 1, 2, 6, 0, 5]
[4, 3, 1, 

[5, 6, 1, 2, 0, 3, 4]
[5, 6, 1, 2, 0, 4, 3]
[5, 6, 1, 2, 3, 0, 4]
[5, 6, 1, 2, 3, 4, 0]
[5, 6, 1, 2, 4, 0, 3]
[5, 6, 1, 2, 4, 3, 0]
[5, 6, 1, 3, 0, 2, 4]
[5, 6, 1, 3, 0, 4, 2]
[5, 6, 1, 3, 2, 0, 4]
[5, 6, 1, 3, 2, 4, 0]
[5, 6, 1, 3, 4, 0, 2]
[5, 6, 1, 3, 4, 2, 0]
[5, 6, 1, 4, 0, 2, 3]
[5, 6, 1, 4, 0, 3, 2]
[5, 6, 1, 4, 2, 0, 3]
[5, 6, 1, 4, 2, 3, 0]
[5, 6, 1, 4, 3, 0, 2]
[5, 6, 1, 4, 3, 2, 0]
[5, 6, 2, 0, 1, 3, 4]
[5, 6, 2, 0, 1, 4, 3]
[5, 6, 2, 0, 3, 1, 4]
[5, 6, 2, 0, 3, 4, 1]
[5, 6, 2, 0, 4, 1, 3]
[5, 6, 2, 0, 4, 3, 1]
[5, 6, 2, 1, 0, 3, 4]
[5, 6, 2, 1, 0, 4, 3]
[5, 6, 2, 1, 3, 0, 4]
[5, 6, 2, 1, 3, 4, 0]
[5, 6, 2, 1, 4, 0, 3]
[5, 6, 2, 1, 4, 3, 0]
[5, 6, 2, 3, 0, 1, 4]
[5, 6, 2, 3, 0, 4, 1]
[5, 6, 2, 3, 1, 0, 4]
[5, 6, 2, 3, 1, 4, 0]
[5, 6, 2, 3, 4, 0, 1]
[5, 6, 2, 3, 4, 1, 0]
[5, 6, 2, 4, 0, 1, 3]
[5, 6, 2, 4, 0, 3, 1]
[5, 6, 2, 4, 1, 0, 3]
[5, 6, 2, 4, 1, 3, 0]
[5, 6, 2, 4, 3, 0, 1]
[5, 6, 2, 4, 3, 1, 0]
[5, 6, 3, 0, 1, 2, 4]
[5, 6, 3, 0, 1, 4, 2]
[5, 6, 3, 0, 2, 1, 4]
[5, 6, 3, 

In [ ]:

## Method 1
x_n = x_0
y_n = y_0
found = dict()
z_it = iter(CyclicPermute(z)) #back to start of cycle

#now map xn and accompanying yn into new palettes and see which combinations work
for z_n in z_it:
    matches = []

    for p1,p2 in palettes:
        p1 = np.array([p1])
        p2 = np.array([p2])

        #mapped values into palette
        x_mapped = p1[np.arange(p1.shape[0])[:,None],x_n]
        y_mapped = p2[np.arange(p2.shape[0])[:,None],y_n]

        idx = np.all(x_mapped + y_mapped == z_n,axis=1)

        #unmapped but fulfilling the criterium
        y_match = y_n[np.ix_(idx)]
        x_match = x_n[np.ix_(idx)] 
        if (len(y_match) >0):
            matches.append([x_match,y_match,p1,p2])
            for x,y in zip(x_match,y_match):
                key = str(x)+str(y)
                res = found.get(key)
                if (res):
                    xp,yp,pp1,pp2 = res
                    pp1.append(p1)
                    pp2.append(p2)
                else:
                    found[key] = (x_match,y_match,[p1],[p2])
            
    #recombine the found working permutations and use them for the next round (basically which x_0's work)
    x_n,y_n = np.zeros((4,),dtype='int'),np.zeros((4,),dtype='int')
    for (xs,ys,ps1,ps2) in matches:
        x_n = np.vstack((x_n,xs))
        y_n = np.vstack((y_n,xs))
    


            
# for (xs,ys,p1,p2) in matches:
#     for x,y in zip(xs,ys):
#         print('x,y:',x,y)
#         x= p1[np.arange(p1.shape[0])[:,None],x]
#         y= p2[np.arange(p2.shape[0])[:,None],y]
        
#         print('p1,p2',p1,p2,'x+y:',x+y,)


2


x: [0 0 2 1 3]
y: [0 1 0 2 3]
pal:
[[[0 0 2 3]
  [0 1 3 3]]

 [[0 1 2 3]
  [0 1 2 3]]

 [[0 2 2 3]
  [0 1 1 3]]

 [[0 3 2 3]
  [0 1 0 3]]]
x...
[0 0 2 2 6]
x...
[[[0 0 1 3]
  [0 3 2 3]]

 [[0 1 1 3]
  [0 3 1 3]]

 [[0 2 1 3]
  [0 3 0 3]]]
x...
[0 0 1 1 6]
x...
[[[0 0 3 3]
  [0 2 1 3]]

 [[0 1 3 3]
  [0 2 0 3]]]
x...
[0 0 3 3 6]
x...


In [751]:
reduced

array([10, 13])

In [601]:
reduced
px[np.ix_(np.arange(len(palettes)),reduced,np.arange(5))].shape
py[np.ix_(np.arange(len(palettes)),reduced,np.arange(5))].shape
# px.shape

(65536, 2, 5)

In [633]:
x_0

array([[0, 0, 0, 1, 2],
       [0, 1, 0, 1, 2],
       [0, 2, 0, 1, 2],
       [1, 0, 0, 1, 2],
       [1, 1, 0, 1, 2],
       [1, 2, 0, 1, 2],
       [0, 0, 1, 1, 2],
       [0, 1, 1, 1, 2],
       [0, 2, 1, 1, 2],
       [1, 0, 1, 1, 2],
       [1, 1, 1, 1, 2],
       [1, 2, 1, 1, 2],
       [0, 0, 2, 1, 2],
       [0, 1, 2, 1, 2],
       [0, 2, 2, 1, 2],
       [1, 0, 2, 1, 2],
       [1, 1, 2, 1, 2],
       [1, 2, 2, 1, 2],
       [0, 0, 3, 1, 2],
       [0, 1, 3, 1, 2],
       [0, 2, 3, 1, 2],
       [1, 0, 3, 1, 2],
       [1, 1, 3, 1, 2],
       [1, 2, 3, 1, 2],
       [0, 0, 0, 2, 2],
       [0, 1, 0, 2, 2],
       [0, 2, 0, 2, 2],
       [1, 0, 0, 2, 2],
       [1, 1, 0, 2, 2],
       [1, 2, 0, 2, 2],
       [0, 0, 1, 2, 2],
       [0, 1, 1, 2, 2],
       [0, 2, 1, 2, 2],
       [1, 0, 1, 2, 2],
       [1, 1, 1, 2, 2],
       [1, 2, 1, 2, 2],
       [0, 0, 2, 2, 2],
       [0, 1, 2, 2, 2],
       [0, 2, 2, 2, 2],
       [1, 0, 2, 2, 2],
       [1, 1, 2, 2, 2],
       [1, 2, 2,